<a href="https://colab.research.google.com/github/veiro/ADS2020/blob/master/prediccion-dataset-imputados/8_Prueba_de_prediccion_categorico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Objetivo:
Utilizar el mejor modelo encontrado para predecir un atributo numerico con el dataset-GAIN

# Configuracion ambiente

In [1]:
USAR_GOOGLE_DRIVE =  True
MODO_DESARROLLO = False

In [2]:
!pip install pycaret
#https://pycaret.gitbook.io/docs/get-started/tutorials

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 483.9/483.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.4/159.4 kB 18.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 76.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 76.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 70.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 97.8 MB/s e

In [3]:
import os
import sys

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from tqdm import tqdm
import pandas
from sklearn.model_selection import train_test_split
from google.colab import drive
#from ydata_profiling import ProfileReport

from datetime import datetime
# para hacer pruebas bajo la cantidad


In [4]:
if (USAR_GOOGLE_DRIVE):
  drive.mount('/content/gdrive')
  PATH_DATA_PROCESADA = "/content/gdrive/MyDrive/Maestria/Tesis/Codigo/data/data-procesada"
else:
  PATH_DATA_PROCESADA = "data/data-procesada"


Mounted at /content/gdrive


# Utils

In [5]:
resultados = pandas.DataFrame()

In [14]:
from pycaret.classification import *

def prediccionCategorico(files):
  global resultados
  ruta_modelo = PATH_DATA_PROCESADA + '/modelo_prediccion/' + 'clsificacion_Country_Region_Desarrollo=False'
  loaded_best_pipeline = load_model(ruta_modelo)

  for fp in  files:
    print(fp)
    data_full = pandas.read_csv(PATH_DATA_PROCESADA+'/'+fp)
    if(MODO_DESARROLLO):
      data =  data_full.sample(n=10000, random_state=1)
    else:
      data = data_full
    # tengo que borrar posibles nulls por generar Country_Region que no existen.
    data_sin_null = data.dropna(subset=['Country_Region'])

    s = setup(data_sin_null, target = 'Country_Region', session_id = 123, use_gpu = True, verbose = False)
    prediction = predict_model(loaded_best_pipeline, data = data_sin_null)
    salida = pull()
    salida.insert(0, "Dataset", fp, allow_duplicates=False)
    resultados = pandas.concat([resultados, salida], axis=0)
    del data_full
    del data
    del s
    del salida
    print("----------------------------------------------------------------")


# Configuracion

In [7]:
files=[
       'csse_covid_19_data_strategy=constant_Desarrollo='+str(MODO_DESARROLLO)+'.csv',
       'csse_covid_19_data_strategy=most_frequent_Desarrollo='+str(MODO_DESARROLLO)+'.csv',
       'csse_covid_19_data_strategy=mean_Desarrollo='+str(MODO_DESARROLLO)+'.csv',
       'csse_covid_19_data_strategy=median_Desarrollo='+str(MODO_DESARROLLO)+'.csv',
       'csse_covid_19_data_strategy=IterativeImputer_Desarrollo='+str(MODO_DESARROLLO)+'.csv',
       'csse_covid_19_data_MIDA_Desarrollo='+str(MODO_DESARROLLO)+'.csv',
       'csse_covid_19_data_GAIN._Desarrollo='+str(MODO_DESARROLLO)+'.csv']

       #'csse_covid_19_data_strategy=KNNImputer_Desarrollo='+str(MODO_DESARROLLO)+'.csv', no lo pude generar

# Prueba de prediccion categorico


In [15]:
%%time
files=[
       'csse_covid_19_data_strategy=constant_Desarrollo='+str(MODO_DESARROLLO)+'.csv',
       'csse_covid_19_data_strategy=most_frequent_Desarrollo='+str(MODO_DESARROLLO)+'.csv',
       'csse_covid_19_data_strategy=mean_Desarrollo='+str(MODO_DESARROLLO)+'.csv',
       'csse_covid_19_data_strategy=median_Desarrollo='+str(MODO_DESARROLLO)+'.csv',
       'csse_covid_19_data_strategy=IterativeImputer_Desarrollo='+str(MODO_DESARROLLO)+'.csv',
       'csse_covid_19_data_MIDA_Desarrollo='+str(MODO_DESARROLLO)+'.csv',
       'csse_covid_19_data_GAIN._Desarrollo='+str(MODO_DESARROLLO)+'.csv']
resultados = pandas.DataFrame()
prediccionCategorico(files)

Transformation Pipeline and Model Successfully Loaded
csse_covid_19_data_strategy=constant_Desarrollo=False.csv
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Warning] There are no meaningful features which satisfy the provided c

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Decision Tree Classifier,0.0234,0.5099,0.0234,0.0077,0.0089,0.0197,0.0583


----------------------------------------------------------------
csse_covid_19_data_strategy=most_frequent_Desarrollo=False.csv
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Warning] There are no meaningful features which satisf

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Decision Tree Classifier,0.0193,0.5082,0.0193,0.0119,0.0109,0.0164,0.0911


----------------------------------------------------------------
csse_covid_19_data_strategy=mean_Desarrollo=False.csv
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Warning] There are no meaningful features which satisfy the pro

...

----------------------------------------------------------------
csse_covid_19_data_MIDA_Desarrollo=False.csv
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Warning] There are no meaningful features which satisfy the provided con

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Decision Tree Classifier,0.0317,0.5111,0.0317,0.0057,0.0090,0.0219,0.0348


----------------------------------------------------------------
csse_covid_19_data_GAIN._Desarrollo=False.csv
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Warning] There are no meaningful features which satisfy the provided co

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Decision Tree Classifier,0.8305,0.9016,0.8305,0.9565,0.8822,0.4984,0.5476


----------------------------------------------------------------
CPU times: user 16min 17s, sys: 33.2 s, total: 16min 50s
Wall time: 16min 41s


In [16]:
resultados

,Dataset,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,csse_covid_19_data_strategy=constant_Desarroll...,Decision Tree Classifier,0.0234,0.5099,0.0234,0.0077,0.0089,0.0197,0.0583
0,csse_covid_19_data_strategy=most_frequent_Desa...,Decision Tree Classifier,0.0193,0.5082,0.0193,0.0119,0.0109,0.0164,0.0911
0,csse_covid_19_data_strategy=mean_Desarrollo=Fa...,Decision Tree Classifier,0.0214,0.5093,0.0214,0.0113,0.0108,0.0185,0.0837
0,csse_covid_19_data_strategy=median_Desarrollo=...,Decision Tree Classifier,0.0211,0.5091,0.0211,0.0126,0.0109,0.0182,0.0948
0,csse_covid_19_data_strategy=IterativeImputer_D...,Decision Tree Classifier,0.0282,0.5122,0.0282,0.0132,0.0135,0.0244,0.1020
0,csse_covid_19_data_MIDA_Desarrollo=False.csv,Decision Tree Classifier,0.0317,0.5111,0.0317,0.0057,0.0090,0.0219,0.0348
0,csse_covid_19_data_GAIN._Desarrollo=False.csv,Decision Tree Classifier,0.8305,0.9016,0.8305,0.9565,0.8822,0.4984,0.5476
